In [1]:
import pandas as pd

playlist_data = pd.read_csv('./cleaned_data/spotify_data/playlistCSV.csv', header=0)
streaming_history = pd.read_csv('./cleaned_data/spotify_data/streamingHistoryCSV.csv', header=0)

In [2]:
merged_dataset = streaming_history.merge(playlist_data, on='trackName',)
merged_dataset['time'] = merged_dataset['time'].str.split(' ').str[1]
merged_dataset['track_id'] = merged_dataset['trackUri'].str.split('track:').str[1]


In [3]:
def month_to_int(value):
    if value == 'June':
        return 6
    elif value == 'July':
        return 7
    elif value == 'August':
        return 8
    else:
        return 9

In [4]:
merged_dataset['Month'] = merged_dataset['Month'].apply(month_to_int)

In [5]:
merged_dataset['date'] = merged_dataset['Month'].astype(str) + '/' + merged_dataset['Day'].astype(str) + '/' + merged_dataset['Year'].astype(str)
merged_dataset['hour'] = merged_dataset['time'].str.split(':').str[0]

In [6]:
merged_dataset = merged_dataset.drop_duplicates(subset='time')
merged_dataset.head()

,artistName_x,Year,Month,Day,time,Sum of msPlayed,trackName,albumName,artistName_y,trackUri,playlists.name,track_id,date,hour
0,"""Weird Al"" Yankovic",2023,6,22,02:00:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,6/22/2023,02
1,"""Weird Al"" Yankovic",2023,7,29,02:38:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,7/29/2023,02
2,"""Weird Al"" Yankovic",2023,7,29,01:10:00,328773,"The Saga Begins (Lyrical Adaption of ""American...",Running With Scissors,"""Weird Al"" Yankovic",spotify:track:74sUbOF9Zm8LdGUJjxleTl,HOmeWorK BANgers,74sUbOF9Zm8LdGUJjxleTl,7/29/2023,01
3,...And You Will Know Us by the Trail of Dead,2023,7,23,01:33:00,325600,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,7/23/2023,01
5,...And You Will Know Us by the Trail of Dead,2023,8,21,03:10:00,25550,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,8/21/2023,03


In [7]:
unique_songs = merged_dataset['trackUri'].unique()
unique_songs = pd.DataFrame(unique_songs)
unique_songs.columns = ['trackUri']
display(unique_songs)

,trackUri
0,spotify:track:5r96TaQquRrlo3Ym3ZlSL2
1,spotify:track:74sUbOF9Zm8LdGUJjxleTl
2,spotify:track:16XbElFPv3A7TrzRJntVYd
3,spotify:track:2SkJKMfjpYsNv0KWOxiegX
4,spotify:track:7dnQZxpbKsHNmtlWcRdsR4
...,...
250,spotify:track:7CskFQLRb7IdIEdnAnjMDV
251,spotify:track:7LygtNjQ65PSdzVjUnHXQb
252,spotify:track:6TSDRzJGwbK9cajVbtqlPV
253,spotify:track:3CkjLJYnnyOtGk4Jx4iYS4


In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from apikey import *

client_id = CLIENT_KEY
client_secret = SECRET_KEY
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [11]:
import time
from tqdm import tqdm

def track_info_progress_bar(unique_songs,api_dataset):
    requests_per_hour = 500
    
    requests_made_hour = 0
    start_time = time.time()
    
    

    for track_uri in tqdm(enumerate(unique_songs['trackUri']), total=len(unique_songs)):
        audio_features = sp.audio_features(track_uri[1])
        new_row = pd.DataFrame(audio_features)
        api_dataset = pd.concat([api_dataset,new_row], ignore_index=True)
        
        requests_made_hour += 1
        elapsed_time = time.time() - start_time
        #time.sleep(60*3)
       
        if requests_made_hour >= requests_per_hour and elapsed_time < 3600:
            time_to_sleep = 3600 - elapsed_time
            time.sleep(time_to_sleep)
            requests_made_hour = 0
            start_time = time.time()

        #tqdm.write(f"Processed: {index + 1}/{end_value}")
    return api_dataset

In [12]:
api_dataset = pd.DataFrame()
api_dataset = track_info_progress_bar(unique_songs,api_dataset)
display(api_dataset)

100%|██████████| 255/255 [00:21<00:00, 11.89it/s]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.728,0.448,8,-10.540,1,0.1720,0.103000,0.0000,0.2670,0.483,80.902,audio_features,5r96TaQquRrlo3Ym3ZlSL2,spotify:track:5r96TaQquRrlo3Ym3ZlSL2,https://api.spotify.com/v1/tracks/5r96TaQquRrl...,https://api.spotify.com/v1/audio-analysis/5r96...,202920,4
1,0.487,0.429,4,-8.716,0,0.0391,0.332000,0.0000,0.2070,0.508,143.582,audio_features,74sUbOF9Zm8LdGUJjxleTl,spotify:track:74sUbOF9Zm8LdGUJjxleTl,https://api.spotify.com/v1/tracks/74sUbOF9Zm8L...,https://api.spotify.com/v1/audio-analysis/74sU...,328773,4
2,0.462,0.806,3,-3.283,1,0.0341,0.260000,0.0000,0.1140,0.354,165.925,audio_features,16XbElFPv3A7TrzRJntVYd,spotify:track:16XbElFPv3A7TrzRJntVYd,https://api.spotify.com/v1/tracks/16XbElFPv3A7...,https://api.spotify.com/v1/audio-analysis/16Xb...,325600,4
3,0.709,0.913,7,-5.148,1,0.0748,0.018200,0.0000,0.1670,0.519,108.024,audio_features,2SkJKMfjpYsNv0KWOxiegX,spotify:track:2SkJKMfjpYsNv0KWOxiegX,https://api.spotify.com/v1/tracks/2SkJKMfjpYsN...,https://api.spotify.com/v1/audio-analysis/2SkJ...,229320,4
4,0.396,0.635,4,-5.665,0,0.0394,0.387000,0.0000,0.2710,0.306,126.981,audio_features,7dnQZxpbKsHNmtlWcRdsR4,spotify:track:7dnQZxpbKsHNmtlWcRdsR4,https://api.spotify.com/v1/tracks/7dnQZxpbKsHN...,https://api.spotify.com/v1/audio-analysis/7dnQ...,178787,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.391,0.828,7,-4.156,1,0.0722,0.000708,0.0000,0.0781,0.561,177.910,audio_features,7CskFQLRb7IdIEdnAnjMDV,spotify:track:7CskFQLRb7IdIEdnAnjMDV,https://api.spotify.com/v1/tracks/7CskFQLRb7Id...,https://api.spotify.com/v1/audio-analysis/7Csk...,217159,4
251,0.472,0.946,2,-4.591,1,0.1450,0.037200,0.0000,0.3520,0.486,177.489,audio_features,7LygtNjQ65PSdzVjUnHXQb,spotify:track:7LygtNjQ65PSdzVjUnHXQb,https://api.spotify.com/v1/tracks/7LygtNjQ65PS...,https://api.spotify.com/v1/audio-analysis/7Lyg...,201027,4
252,0.714,0.721,9,-5.049,1,0.0783,0.273000,0.2450,0.1740,0.415,83.021,audio_features,6TSDRzJGwbK9cajVbtqlPV,spotify:track:6TSDRzJGwbK9cajVbtqlPV,https://api.spotify.com/v1/tracks/6TSDRzJGwbK9...,https://api.spotify.com/v1/audio-analysis/6TSD...,164093,4
253,0.219,0.952,2,-3.621,1,0.1760,0.000051,0.0989,0.1570,0.172,112.886,audio_features,3CkjLJYnnyOtGk4Jx4iYS4,spotify:track:3CkjLJYnnyOtGk4Jx4iYS4,https://api.spotify.com/v1/tracks/3CkjLJYnnyOt...,https://api.spotify.com/v1/audio-analysis/3Ckj...,176307,1


In [13]:
api_dataset.to_csv('./cleaned_data/mergedSpotifyData.csv', index=False)

In [14]:
columns_to_drop = ['analysis_url','track_href','type','uri']
api_dataset.drop(columns_to_drop, axis=1,inplace=True)
api_dataset.rename(columns={'id': 'track_id'}, inplace=True)


In [15]:
spotify_dataset = merged_dataset.merge(api_dataset, on='track_id')
columns_to_drop = ['trackUri','Sum of msPlayed','artistName_x']
spotify_dataset.drop(columns_to_drop, axis=1,inplace=True)
spotify_dataset.rename(columns={'artistName_y': 'artistName'}, inplace=True)
spotify_dataset.to_csv('apiData.csv', index=False)
display(spotify_dataset)

,Year,Month,Day,time,trackName,albumName,artistName,playlists.name,track_id,date,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2023,6,22,02:00:00,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,6/22/2023,...,-10.540,1,0.1720,0.103000,0.0000,0.2670,0.483,80.902,202920,4
1,2023,7,29,02:38:00,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,7/29/2023,...,-10.540,1,0.1720,0.103000,0.0000,0.2670,0.483,80.902,202920,4
2,2023,7,29,01:10:00,"The Saga Begins (Lyrical Adaption of ""American...",Running With Scissors,"""Weird Al"" Yankovic",HOmeWorK BANgers,74sUbOF9Zm8LdGUJjxleTl,7/29/2023,...,-8.716,0,0.0391,0.332000,0.0000,0.2070,0.508,143.582,328773,4
3,2023,7,23,01:33:00,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,Missing Her,16XbElFPv3A7TrzRJntVYd,7/23/2023,...,-3.283,1,0.0341,0.260000,0.0000,0.1140,0.354,165.925,325600,4
4,2023,8,21,03:10:00,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,Missing Her,16XbElFPv3A7TrzRJntVYd,8/21/2023,...,-3.283,1,0.0341,0.260000,0.0000,0.1140,0.354,165.925,325600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,2023,6,8,11:32:00,Heart and Soul,Great Divide (Deluxe Version),Twin Atlantic,Missing Her,7CskFQLRb7IdIEdnAnjMDV,6/8/2023,...,-4.156,1,0.0722,0.000708,0.0000,0.0781,0.561,177.910,217159,4
1297,2023,8,30,15:43:00,Cherry Pie,Cherry Pie,Warrant,Rock Paper Scissors,7LygtNjQ65PSdzVjUnHXQb,8/30/2023,...,-4.591,1,0.1450,0.037200,0.0000,0.3520,0.486,177.489,201027,4
1298,2023,7,23,08:29:00,Que Sera,Tales of the Forgotten Melodies,Wax Tailor,Missing Her,6TSDRzJGwbK9cajVbtqlPV,7/23/2023,...,-5.049,1,0.0783,0.273000,0.2450,0.1740,0.415,83.021,164093,4
1299,2023,8,12,18:04:00,Woman,Wolfmother,Wolfmother,Rock Paper Scissors,3CkjLJYnnyOtGk4Jx4iYS4,8/12/2023,...,-3.621,1,0.1760,0.000051,0.0989,0.1570,0.172,112.886,176307,1
